In [83]:
import pandas as pd
import duckdb
import os

In [334]:
files = os.listdir('data')
files = sorted([f for f in files if f.endswith('.csv')])[1:]
files = sorted(files, reverse=True)

In [358]:
def clean_string(s):
    # Convert to lowercase
    s = s.lower()
    # Replace spaces with underscores
    s = s.replace(" ", "_")
    # Remove special characters
    s = ''.join(e for e in s if e.isalnum() or e == '_')
    # Replace accented characters
    accents = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Á': 'a', 'É': 'e', 'Í': 'i', 'Ó': 'o', 'Ú': 'u',
        'ñ': 'n', 'Ñ': 'n'
    }
    for accent, replacement in accents.items():
        s = s.replace(accent, replacement)
    return s

listOfDates = [i.split(".")[0] for i in files]

procc_list = []

for index_file, read_file in enumerate(files):
    # print(listOfDates[index_file])
    # print(read_file)
    procc_file = duckdb.sql(f"""
            WITH base as (SELECT 
            CASE WHEN "Entidad Federativa" = 'DISTRITO FEDERAL' THEN 'CIUDAD DE MÉXICO' ELSE "Entidad Federativa" END as estado,
            "Facultad o Escuela de Medicina" as facultad,
            "Sustentante" as sustentante,
            "Seleccionado" as seleccionado,
            "Promedio General de Conocimientos Médicos" as promedio,
            ROUND(("Seleccionado"::INTEGER / "Sustentante"::INTEGER) * 100,2) as acceptance_rate,
            '{listOfDates[index_file]}' as date_id,
            '{listOfDates[index_file].split('_')[1]}' as year,
            FROM read_csv('data/{read_file}') WHERE "Sustentante" > 0 ),
            stats AS (
                SELECT 
                    AVG(acceptance_rate) AS mean_rate, 
                    STDDEV(acceptance_rate) AS stddev_rate 
                FROM base
            ),
            enhanced AS (
                SELECT *,
                    mean_rate,
                    stddev_rate,
                    ROUND(acceptance_rate + stddev_rate,2) AS acceptance_rate_plus_1_std,
                    ROUND(acceptance_rate - stddev_rate,2) AS acceptance_rate_minus_1_std,
                    ROUND((acceptance_rate - mean_rate) / stddev_rate,2) AS z_index
                FROM base, stats
            )
            SELECT * EXCLUDE (mean_rate,mean_rate_1,stddev_rate_1, stddev_rate)
            FROM enhanced 
        """).df()
    
    procc_list.append(procc_file)

dataframe_full = pd.concat(procc_list, ignore_index=True)

## On the column estado in the dataframe_full if there are None values replace them with the string "Otro"
dataframe_full['estado'] = dataframe_full['estado'].fillna('OTRA')
dataframe_full['estado_id'] = dataframe_full['estado'].apply(clean_string)
dataframe_full['school_id'] = dataframe_full['facultad'].apply(clean_string)

dataframe_full['sustentante'] = dataframe_full['sustentante'].fillna(0)
dataframe_full['sustentante'] = dataframe_full['sustentante'].astype(int)
dataframe_full['seleccionado'] = dataframe_full['seleccionado'].fillna(0)
dataframe_full['seleccionado'] = dataframe_full['seleccionado'].astype(int)

In [326]:
listOfSchools = duckdb.sql("""SELECT 
           school_id, COUNT(*) as apprs 
           FROM dataframe_full 
           GROUP BY school_id 
           HAVING apprs = 1
           ORDER BY school_id DESC""").df()['school_id'].to_list()

In [327]:
for school_id in listOfSchools:
  
  print(duckdb.sql(f"""
                   WITH base AS (
    SELECT estado,
          facultad,
          school_id,
          year::INTEGER,
          date_id,
          year::INTEGER - LEAD(year::INTEGER) OVER (PARTITION BY school_id ORDER BY year::INTEGER DESC) AS year_gap
      FROM dataframe_full 
      WHERE school_id = '{school_id}'
      ORDER BY year DESC)
      FROM base
      """))

┌────────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────┬─────────────────────────┬────────────┬──────────┐
│   estado   │                  facultad                   │                  school_id                  │ CAST("year" AS INTEGER) │  date_id   │ year_gap │
│  varchar   │                   varchar                   │                   varchar                   │          int32          │  varchar   │  int32   │
├────────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────────┼────────────┼──────────┤
│ TAMΔΙ ΊΡΑΣ │ UNIVERSIDAD DEL VALLE DE MEXICO CD VICTORIA │ universidad_del_valle_de_mexico_cd_victoria │                    2014 │ enarm_2014 │     NULL │
└────────────┴─────────────────────────────────────────────┴─────────────────────────────────────────────┴─────────────────────────┴────────────┴──────────┘

┌────────────┬───────────────────────────────────────────

In [331]:
duckdb.sql("""
    SELECT 
        *,
        ROUND(
            (acceptance_rate - LAG(acceptance_rate) OVER (PARTITION BY school_id ORDER BY year)) 
            / NULLIF(LAG(acceptance_rate) OVER (PARTITION BY school_id ORDER BY year), 0), 
            2
        ) * 100 AS yoy_growth
    FROM dataframe_full 
    WHERE school_id = 'inst_est_tec_ysup_de_matatipac_tepiq'
    ORDER BY year DESC
    """)


┌─────────┬───────────────────────────────────────────┬─────────────┬──────────────┬──────────┬─────────────────┬────────────┬─────────┬────────────────────────────┬─────────────────────────────┬─────────┬───────────┬──────────────────────────────────────┬────────────┐
│ estado  │                 facultad                  │ sustentante │ seleccionado │ promedio │ acceptance_rate │  date_id   │  year   │ acceptance_rate_plus_1_std │ acceptance_rate_minus_1_std │ z_index │ estado_id │              school_id               │ yoy_growth │
│ varchar │                  varchar                  │    int64    │    int64     │ varchar  │     double      │  varchar   │ varchar │           double           │           double            │ double  │  varchar  │               varchar                │   double   │
├─────────┼───────────────────────────────────────────┼─────────────┼──────────────┼──────────┼─────────────────┼────────────┼─────────┼────────────────────────────┼─────────────────────────

In [151]:
sorted(duckdb.sql(""" SELECT DISTINCT school_id FROM dataframe_full """).df().drop_duplicates()['school_id'].to_list())

['b_univ_aut_puebla_puebla',
 'b_univ_aut_puebla_tehuacan',
 'b_univ_aut_puebla_teziutlan',
 'centro_est_univ_xochicalco_campus_mexicali',
 'centro_est_univ_xochicalco_u_ensenada',
 'centro_est_univ_xochicalco_u_tijuana',
 'el_campus_universitario_siglo_xxi',
 'esc_de_medicina_intermedica',
 'esc_iberoamericana_de_ciencias_de_la_salud',
 'esc_libre_de_homeopatia_de_mexico',
 'esc_med_tominaga_nakamoto',
 'esc_medico__militar_sedena',
 'esc_medico__naval_semar',
 'estudios_en_el_extranjero',
 'inst_de_ciencias_y_estudios_sup_de_tamaulipas',
 'inst_est_superiores_de_chiapas',
 'inst_est_superiores_de_chiapas__c_ampus_tapachula',
 'inst_est_superiores_de_chiapas__campus_tapachula',
 'inst_est_superiores_de_chiapas__campus_tuxtla_gutierrez',
 'inst_est_superiores_de_chiapas_campus_tuxtla_gutierrez',
 'inst_est_superiores_de_chiapascampus_tuxtla_gutierrez',
 'inst_est_superiores_de_istmo_de_tehuantepec',
 'inst_est_superiores_tepeaca',
 'inst_est_tec_y_sup_de_matatipac_tepic',
 'inst_est_te